# Super-structure scenario modelling in brightway

This notebook shows how to use the bw_scenarios module, using the example of a bike.

We start by importing an SDF file (.xslx or .csv).

## Project setup

In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc

In [2]:
PROJECT_NAME = "bc24-hackathon-bike-project"
BIO_DB_NAME = "ecoinvent-3.10-biosphere"

In [3]:
bd.projects.set_current(PROJECT_NAME)
bd.databases

Databases dictionary with 3 object(s):
	bike_production_example
	biosphere3
	ecoinvent-3.10-biosphere

In [4]:
xl_importer = bi.importers.ExcelImporter("bike_production_example.xlsx")

Extracted 1 worksheets in 1.48 seconds


In [5]:
for data_item in xl_importer.data:
    for exchange in data_item["exchanges"]:
        if exchange["type"] == "biosphere" and exchange["database"] == "biosphere3":
            print(f"Update bio db name for {exchange}")
            exchange["database"] = BIO_DB_NAME

In [6]:
xl_importer.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 18.32 seconds


In [7]:
xl_importer.statistics()

4718 datasets
	0 exchanges
	0 unlinked exchanges (0 unique)
	


(4718, 0, 0, 0)

In [8]:
if xl_importer.statistics(print_stats=False)[2] == 0:
    xl_importer.write_database()

100%|██████████| 4718/4718 [00:00<00:00, 6059.73it/s]


Vacuuming database 
Created database: biosphere3


In [9]:
bike_db = bd.Database("bike_production_example")
for a in bike_db:
    print(f"{a} -> {a['code']}")

'natural gas production' (MJ, NO, None) -> ng
'carbon fibre production' (kg, DE, None) -> cf
'bike production' (bike, DK, None) -> bike


In [10]:
import sys
sys.path.append("../")
from pathlib import Path
from functools import partial

import bw2data as bd

from bw_scenarios import Scenario, scenarios, SDFImporter
from bw_scenarios.strategies import link_scenario_on_keys, separate_keys, replace_field

In [11]:
file_path = Path("sdf.xlsx")

# store the scenario data in a variable
importer = SDFImporter.from_excel(file_path)
importer.unlinked

,from activity name,from reference product,from location,from categories,from database,from key,to activity name,to reference product,to location,to categories,...,to key,flow type,default,regular case,change 1 sibling,change all sibling,change production,change production and sibling,from id,to id
0,carbon fibre production,carbon fibre,DE,NaN,bike_production_example,"('bike_production_example', '7bde0c388a1b401c9...",carbon fibre production,carbon fibre,DE,NaN,...,"('bike_production_example', '7bde0c388a1b401c9...",production,1.00,1.00,1.00,1.00,1.20,1.200,None,None
1,natural gas production,natural gas,NO,NaN,bike_production_example,"('bike_production_example', '6ba895871c694e9a8...",carbon fibre production,carbon fibre,DE,NaN,...,"('bike_production_example', '7bde0c388a1b401c9...",technosphere,237.00,200.00,237.00,237.00,237.00,237.000,None,None
2,carbon fibre production,carbon fibre,DE,NaN,bike_production_example,"('bike_production_example', '7bde0c388a1b401c9...",carbon fibre production,carbon fibre,DE,NaN,...,"('bike_production_example', '7bde0c388a1b401c9...",technosphere,0.01,0.01,0.01,0.01,0.01,0.005,None,None
3,carbon fibre production,carbon fibre,DE,NaN,bike_production_example,"('bike_production_example', '7bde0c388a1b401c9...",production of bike,bike,DK,NaN,...,"('bike_production_example', '1d31f61bf8884a6fa...",technosphere,2.00,2.00,1.90,1.90,2.00,2.000,None,None
4,carbon fibre production,carbon fibre,DE,NaN,bike_production_example,"('bike_production_example', '7bde0c388a1b401c9...",production of bike,bike,DK,NaN,...,"('bike_production_example', '1d31f61bf8884a6fa...",technosphere,0.50,0.50,0.50,0.40,0.50,0.500,None,None
5,"Carbon dioxide, fossil",NaN,NaN,"('air',)",biosphere3,"('biosphere3', '349b29d1-3e58-4c66-98b9-9d1a07...",carbon fibre production,carbon fibre,DE,NaN,...,"('bike_production_example', '7bde0c388a1b401c9...",biosphere,26.60,13.00,26.60,26.60,26.60,26.600,None,None


In [12]:
importer.apply_strategy(separate_keys)
importer.apply_strategy(partial(replace_field, field="to database", replace={"biosphere3": BIO_DB_NAME}))
importer.apply_strategy(partial(replace_field, field="from database", replace={"biosphere3": BIO_DB_NAME}))
importer.apply_strategy(link_scenario_on_keys)

importer.unlinked

,from activity name,from reference product,from location,from categories,from database,to activity name,to reference product,to location,to categories,to database,...,default,regular case,change 1 sibling,change all sibling,change production,change production and sibling,from id,to id,from code,to code


In [13]:
importer.write()
scenarios

Scenarios dictionary with 5 object(s):
	change 1 sibling
	change all sibling
	change production
	change production and sibling
	regular case

In [14]:
for scenario in scenarios:
    Scenario(scenario).add_datapackage()
Scenario("change 1 sibling").exchanges

{4365: [(4365, 'production', 1.0),
  (4364, 'technosphere', 237.0),
  (4365, 'technosphere', 0.01),
  (5240, 'biosphere', 26.6)],
 4363: [(4365, 'technosphere', 1.9), (4365, 'technosphere', 0.5)]}

In [17]:
bike = bd.get_node(name="bike production")
method = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')

In [18]:
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {bike: 1},
    method=method
)
data_objs

In [19]:
for scenario in scenarios:
    my_lca = bc.LCA({bike.id:1}, data_objs=data_objs + [Scenario(scenario).dp_static])
    my_lca.lci()
    my_lca.lcia()
    print(my_lca.score)

67.17171813502456
64.48484940962358
61.79798068422261
55.88235374258346
55.64853636290738
